# WeightedRandomSampler

In [24]:
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
s = np.array(list(WeightedRandomSampler([0.1, 0.9, 0.4, 3.0, 0.6], 20000, replacement=True)))
from collections import Counter
counter = Counter(s)
print(counter)
for k,v in counter.items():
    counter[k] = v/20000
for i in range(0, 5):
    print(i, counter[i])

Counter({3: 11859, 1: 3642, 4: 2423, 2: 1623, 0: 453})
0 0.02265
1 0.1821
2 0.08115
3 0.59295
4 0.12115


In [31]:
import pandas as pd
folds = pd.read_csv('./trained-models/sgd_onecycle_bnfrozen_smoothed/folds.csv')
values = folds.label.values

classcounts = Counter(values)
classcounts

Counter({3: 11167, 4: 2206, 1: 1838, 2: 2028, 0: 948})

In [93]:
train_df = folds.iloc[folds[folds.fold != 0].index].reset_index(drop=True)
print(len(train_df))
target = train_df.label.values
print('target classes', np.unique(target))

from sklearn.utils.class_weight import compute_class_weight
w = compute_class_weight({0:3,
                          1:2.,
                          2:2.3,
                          4:2.3}, np.unique(target), target)


class_sample_count = np.unique(target, return_counts=True)[1]
print(class_sample_count)
class_sample_count[0] *= 3
class_sample_count[1] *= 2
class_sample_count[2] *= 2.3
class_sample_count[4] *= 2.3

print(class_sample_count)

weight = 1. / class_sample_count
print('w', weight)
samples_weight = weight[target] # unpacks
print(len(samples_weight))
samples_weight = torch.from_numpy(samples_weight)

sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

sampled = np.array(list(sampler))
sampled

14549
target classes [0 1 2 3 4]
[ 758 1470 1623 8933 1765]
[2274 2940 3732 8933 4059]
w [0.00043975 0.00034014 0.00026795 0.00011194 0.00024637]
14549


/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[3 4 4 ... 4 1 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


array([ 8970,  7042,  2124, ...,  2022, 13297,   599])

# smooth cross entropy

In [53]:
a = np.array([[1,2,3],
             [4,5,6]])
a.sum(axis=0), a.sum(axis=1)

(array([5, 7, 9]), array([ 6, 15]))

In [52]:
x = torch.tensor([[0.5, 0.25, 0.75],
                  [0.2, 0.7, 0.1],
                  [0.1, 0.1, 0.8]])
target = torch.tensor([0,2,1])

weight = torch.zeros_like(x)
weight.fill_(0.05 / (3 - 1))
print(weight) # defaults

# randomly scatter self.confidence at indexes across each col dimension
print(weight.scatter_(1, target.unsqueeze(1), 0.95))

# each row contains confidences for the given sample
# each column is  a class

dist_lsm_input = -weight * x
torch.mean(torch.sum(dist_lsm_input, dim=1)) # sum across the columns (each sample's predictions)

tensor([[0.0250, 0.0250, 0.0250],
        [0.0250, 0.0250, 0.0250],
        [0.0250, 0.0250, 0.0250]])
tensor([[0.9500, 0.0250, 0.0250],
        [0.0250, 0.0250, 0.9500],
        [0.0250, 0.9500, 0.0250]])


tensor(-0.2450)

In [96]:
from loss_functions import bi_tempered_logistic_loss 

device = "cpu"

activations = torch.FloatTensor([[-0.5,  0.1,  2.0],
                                [0.1,2,3]]).to(device)
labels = torch.FloatTensor([[0.2, 0.5, 0.3],
                           [0.1,0.2,0.3]]).to(device)

# The standard logistic loss is obtained when t1 = t2 = 1.0
loss = bi_tempered_logistic_loss(activations=activations, labels=labels, t1=1.0, t2=1.0)
print("Loss, t1=1.0, t2=1.0: ", loss)

loss = bi_tempered_logistic_loss(activations=activations, labels=labels, t1=0.7, t2=1.3)
print("Loss, t1=0.7, t2=1.3: ", loss)

torch.Size([2, 3]) torch.Size([2, 1])
torch.Size([2])
Loss, t1=1.0, t2=1.0:  tensor([0.6287, 0.1883])
logt torch.Size([2, 1])
point torch.Size([2, 1])
torch.Size([2, 3]) torch.Size([2, 1])
torch.Size([2])
Loss, t1=0.7, t2=1.3:  tensor([0.2630, 0.0919])
